# Importing Libraries

In [55]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go 
from ipywidgets import interact
import ipywidgets as wg
%matplotlib inline 

In [56]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

<IPython.core.display.Javascript object>

# Initialize Parameters

In [57]:
# initialize parameters values
def init_param(f):
        x1 =0.2254
        x2= 0.3762 
        x3 = 1 
        Mu = 0.1515
        Eta=1/7
        Delta=1/3
        #data start from Jan. 28 until August 12.
        RealDataDR =[0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,62,53,97,93,78,250,238,240,561,347,466,587,769,778,
                         1247,1492,1797,977,2313,2651,2547,3497,
                         2823,4000,3526,4207,5322,5986,6557,5560,4789,5249,5210,6153,5959,5974,5217,4050,4053,4782,4668,4585,
                         4805,4316,3599,3039,3836,4204,3951,4694,4092,3153,2972,2667,3786,3493,3491,3047,2256,2729,3370,2646,
                         3021,2357,2324,1739,2091,2086,1872,1965,1900,1389,1221,1075,1444,1401,1327,1083,802,744,1402,888,992,
                         789,875,675,451,813,665,642,652,669,531,300,397,584,593,516,416,355,178,318,321,177,
                         518,270,197,280,283,202,379,163,346,338,301,210,328,331,148,264,224,221,113,577,296,255,175,174,126,142,
                         182,201,223,235,192,208,137,193,214,276,188,234,169,114,162,230,231,249,218,190,12,280,306,252,274,254,
                         168,202,288,382,379,295,238,159,190,384,401,552,347,463,259,412]
        RealDataCum=[3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,17,79,132,229,322,400,650,888,1128,1689,2036,2502,3089,3858,4636,5883,
                         7375,9172,10149,12462,15113,17660,21157,23980,27980,31506,35713,41035,47021,
                         53578,59138,63927,69176,74386,80539,86498,92472,97689,101739,105792,110574,
                         115242,119827,124632,128948,132547,135586,139422,143626,147577,152271,156363,
                         159516,162488,165155,168941,172434,175925,178972,181228,183957,187327,189973,
                         192994,195351,197675,199414,201505,203591,205463,207428,209328,210717,211938,
                         213013,214457,215858,217185,218268,219070,219814,221216,222104,223096,223885,
                         224760,225435,225886,226699,227364,228006,228658,229327,229558,23015,230555,
                         231139,231732,232248,232664,233019,233197,233515,233836,234013,234531,234801,
                         234998,235278,235561,235763,236142,236305,236651,236989,237290,237500, 237828,
                         238159,238011,238275,238499,238720,238833,239410,239706,239961,240136,
                         240310,240436,240578,240760,240961,241184,241419,241611,241819,241956,242149,
                         242363,242639,242827,243061,243230,243344,243506,24336,243967,244216,244434,244624,
                         244752,245032,245338,245590,245864,246118,246286,246488,246776,247158,247537,247832,248070,248229,
                         248419,248803,249204,249756,250103,250566,250825,251237]
        S0 = 60.48e6; #number of susceptible at time t0
        N = 46
        v = 1/27;
        Alpha= 1/14;
        v1 = f*v;
        v2 = (1-f)*v;
        t0 = (1/x2) - math.log(x1);          #initial time for the beginning of the early exponential growth phase
        return N, t0, v, Alpha, v1, v2, f, x1, x2, x3, Mu, RealDataDR, RealDataCum, S0,Eta,Delta




# Common Functions

In [58]:
# calculate Initial population in the I and U compartments
def Cal_Init_I0U0(S0,x3,x2,f,v1,v2,Alpha,Eta):
    I0 = (x3*x2)/(f*(v1+v2));                        #number of asymptomatic infectious at time t0
    E0 = ((x2+v1+v2)*I0) / Alpha;               #number of asymptomatic non-infectious at time t0
    U0 = I0*(v2/(Eta+x2));   #number of unreported symptomatic infectious at time t0
    return I0, E0, U0, S0

# transmission rate at time t0
def Ti_0(x2, v1, v2, S, Eta, Delta):
    return ((x2 + v1 + v2) / S)*((Eta + x2) / (v2 + Eta + x2)) * (math.exp(x2 * Delta))


# time dependent transmission rate
def Ti(t, N, x2, v1, v2, S, Alpha, Mu): 
    if t <= N:
        retval = ((x2 + v1 + v2) / S)*((Alpha + x2) / (v2 + Alpha + x2))
    else:
        retval = (((x2 + v1 + v2) / S)*((Alpha + x2) / (v2 + Alpha + x2)))* math.exp(-Mu*(t - N))
    return retval


def S_delta(S, I, E, U, t, N, x2, v1, v2,  Alpha, Mu):
    return (Ti(t, N, x2, v1, v2, S, Alpha, Mu))


def E_delta(S, I, E, U, t, N, x2, v1, v2,  Alpha, Mu):
    return (Ti(t, N, x2, v1, v2, E, Alpha, Mu))


def I_delta(S, I, E, U, t, N, x2, v1, v2,  Alpha, Mu):
    return (Ti(t, N, x2, v1, v2, I, Alpha, Mu))


def U_delta(S, I, E, U, t, N, x2, v1, v2,  Alpha, Mu):
    return (Ti(t, N, x2, v1, v2, U, Alpha, Mu))


def lat_per(S, E, I, U, t, N, x2, v1, v2, Alpha, Mu, Delta): 
    if t <= Delta:
        T_del=Ti(t, N, x2, v1, v2, S, Alpha, Mu)
        S_del=S_delta(S, I, E, U, t, N, x2, v1, v2,  Alpha, Mu)
        E_del=E_delta(S, I, E, U, t, N, x2, v1, v2,  Alpha, Mu)
        I_del=I_delta(S, I, E, U, t, N, x2, v1, v2,  Alpha, Mu)
        U_del=U_delta(S, I, E, U, t, N, x2, v1, v2,  Alpha, Mu)
        dEdt = T_del * S_del *(I_del + U_del)- (Alpha * E_del)
        dIdt = (Alpha * E_del) - ((v1+v2)* I_del) 
    else:
        t1=t-Delta
        T_del=Ti(t1, N, x2, v1, v2, S, Alpha, Mu)
        S_del=S_delta(S, I, E, U, t1, N, x2, v1, v2,  Alpha, Mu)
        E_del=E_delta(S, I, E, U, t1, N, x2, v1, v2,  Alpha, Mu)
        I_del=I_delta(S, I, E, U, t1, N, x2, v1, v2,  Alpha, Mu)
        U_del=U_delta(S, I, E, U, t1, N, x2, v1, v2,  Alpha, Mu)
        dEdt = T_del * S_del *(I_del + U_del)- (Alpha * E_del)
        dIdt = (Alpha * E_del) - ((v1+v2)* I)    
    return dEdt,dIdt

# transmission rate over time
def Ti_over_time( N, x2, v1, v2, S, Alpha, Mu):
    t = np.linspace(0, 100, 100)     # Grid of time points (in days)
    return[Ti(i, N, x2, v1, v2, S, Alpha, Mu) for i in range(len(t))]
    
# basic reproduction number at time t0
def R_0(ti0, v1, v2, S, Eta):
    return ((ti0 * S)/(v1 + v2)) * (1 + (v2 / Eta))


# Cumulative Data

In [59]:
# calculating cumulative reported and unreported
def CumRep(t,t0,dIdt,N,x1,x2,x3):
    if t <= N:
        retval = x1*math.exp(x2*t)-x3
    else:
        retval = dIdt
    return retval

# differential equation
def deriv_Cum(y, t, Ti, v, Alpha, v1, v2 ,f, t0, N, x2, Mu,x1,Eta,Delta ):
    S, E, I, U, CR, CU = y
    dSdt = -Ti(t, N, x2, v1, v2, S, Alpha, Mu) * S *(I + U)
    dEdt,dIdt = lat_per(S, E, I, U, t, N, x2, v1, v2, Alpha, Mu, Delta)
    dUdt = (v2 * I) - (Eta * U)
    CR = I
    CR += CumRep(t, t0, dIdt,N,x1,x2,1)
    CU = I + U
    CU += CumRep(t, t0, dIdt,N,x1,x2,1)
    return dSdt, dEdt, dIdt, dUdt, CR, CU


def Calc_Cum(S0, x3, x2, v, Alpha, v1, v2, f, t0, N, Mu,x1,Eta,Delta):
    # Initial population in the different compartments
    E0, I0, U0, S0 = Cal_Init_I0U0(S0,x3,x2,f,v1,v2, Alpha, Eta)
    CR0 = 1.0   #number of cumulated reported symptomatic infectious at time t0
    CU0 = 1.0   #number of cumulated unreported symptomatic infectious at time t0
    
    t = np.linspace(0, 100, 100)     # Grid of time points (in days)
    y0 = [S0, E0, I0, U0, CR0, CU0]    #initial population in different compartments
    print ('I0 = ' + "{:.2f}".format(I0))
    print ('U0 = ' + "{:.2f}".format(U0))
    print('N = ' + str(N))
    # Integrate the SEIR equations over the time grid, t.
    ret = odeint(deriv_Cum, y0, t, args=(Ti, v, Alpha, v1, v2, f, t0, N, x2, Mu,x1, Eta,Delta ))
    S, E, I, U, CR, CU= ret.T
    CR = v1 * CR
    CU = v2 * CU
    return t, CR, CU

# Daily Reported Data

In [60]:
# differential equation
def deriv_Daily(y, t, Ti, v, Alpha, v1, v2 ,f, N, x2, Mu,Eta,Delta ):
    S, E, I,  U ,DR = y
    dSdt = -Ti(t, N, x2, v1, v2, S, Alpha, Mu) * S *(I + U)
    dEdt,dIdt = lat_per(S, E, I, U, t, N, x2, v1, v2, Alpha, Mu, Delta)
    dUdt = (v2 * I) - (Eta * U)
    dDRdt = (v1 * I) - (Eta  * DR)
    return dSdt, dEdt, dIdt, dUdt ,dDRdt

def Calc_Daily(S0, x3, x2, v, Alpha, v1, v2, f, RealDataDR, N, Mu, Eta,Delta ):
    # Initial population in the E, I and U compartments
    E0, I0, U0, S0 = Cal_Init_I0U0(S0,x3,x2,f,v1,v2,Alpha,Eta)
    DR0 = 1.0   #daily number of reported symptomatic infectious at time t0

    t = np.linspace(0, 100, 100)     # Grid of time points (in days)
    y0 = [S0, E0, I0, DR0, U0]            #initial population in different compartments
    
    # Integrate the SEIRU equations over the time grid, t.
    ret = odeint(deriv_Daily, y0, t, args=(Ti, v, Alpha, v1, v2,f, N, x2, Mu,Eta,Delta ))
    S, E, I,  U, DR = ret.T
    
    return t, DR, RealDataDR

# Epidemic Curve

In [61]:
# differential equation
def deriv(y, t, Ti, v, Alpha, v1, v2, N, x2, Mu,Eta,Delta ):
    S, E,I, R, U  = y
    dSdt = -Ti(t, N, x2, v1, v2, S, Alpha, Mu) * S *(I + U)
    dEdt,dIdt = lat_per(S, E, I, U, t, N, x2, v1, v2, Alpha, Mu, Delta)
    dRdt = (v1 * I) - (Eta  * R)
    dUdt = (v2 * I) - (Eta * U)
    return dSdt,dEdt, dIdt, dRdt, dUdt 

def Calc_All(S0, x3, x2, v, Alpha, v1, v2, f, N, Mu,Eta,Delta ):
    # Initial population in the I and U compartments
    E0, I0, U0, S0 = Cal_Init_I0U0(S0,x3,x2,f,v1,v2,Alpha,Eta )
    R0 = 1.0   # number of reported symptomatic infectious at time t0
    
    t = np.linspace(0, 100, 100)     # Grid of time points (in days)
    y0 = [S0, E0, I0, R0, U0]            #initial population in different compartments

    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(Ti, v, Alpha, v1, v2, N, x2, Mu,Eta,Delta ))
    S, E, I, R, U = ret.T
    return t, S, E, I, R, U

## Plotting the solution

In [62]:
def plotsiru(f):
    #initialize parameters based on the country
    N, t0, v, Alpha, v1, v2, f, x1, x2, x3, Mu, RealDataDR, RealDataCum, S0,Eta,Delta= init_param(f)
    
    #Cumulative graph
    figCum = go.Figure()
    #solving differential equations
    t, CR, CU = Calc_Cum(S0, x3, x2, v, Alpha, v1, v2, f, t0, N, Mu, x1, Eta,Delta)
    t, S, E, I, R, U = Calc_All(S0, x3, x2, v, Alpha, v1, v2, f, N, Mu, Eta,Delta)
    figCum.add_trace(go.Scatter(x=t, y=I, name='I(t)', mode = "lines", line = dict(color = "Red", shape = "spline", smoothing = 1)))
    figCum.add_trace(go.Scatter(x=t, y=R, name='R(t)', mode = "lines", line = dict(color = "blue", shape = "spline", smoothing = 1)))
    figCum.add_trace(go.Scatter(x=t, y=U, name='U(t)', mode = "lines", line = dict(color = "green", shape = "spline", smoothing = 1)))
   
    
    figCum.add_trace(go.Scatter(x=t, y=CR, name='CR(t)', mode = "lines", line = dict(color = "blue", shape = "spline", smoothing = 1))) 
    figCum.add_trace(go.Scatter(x=t, y=CU, name='CU(t)', mode = "lines", line = dict(color = "green", shape = "spline", smoothing = 1)))
    figCum.add_trace(go.Scatter(x=t, y=RealDataCum, name='Cumulative Data', mode = "markers", line = dict(color = "black", shape = "spline", smoothing = 1)))
    figCum.update_layout(autosize=False, width= 900, height=500, title='Cumulative Data ({})', yaxis_title='Individuals')
    
    #Daily reported cases graph
    figDaily = go.Figure()
    t, DR, RealDataDR = Calc_Daily(S0, x3, x2, v, Alpha, v1, v2, f, RealDataDR, N, Mu, Eta,Delta )
    figDaily.add_trace(go.Scatter(x=t, y=DR, name='DR(t)', mode = "lines", line = dict(color = "purple", shape = "spline", smoothing = 1)))
    figDaily.add_trace(go.Scatter(x=t, y=RealDataDR, name='Daily Data', mode = "markers", line = dict(color = "black", shape = "spline", smoothing = 1)))
    #figDaily.add_trace(go.Bar(x=t, y=RealDataDR, name='Daily Data'))
    figDaily.update_layout(autosize=False, width= 900, height=500,title='Daily Data ({})', yaxis_title='Individuals')
    
    
   
    
    #transmission rate over time
    ti_overtime = Ti_over_time(N, x2, v1, v2, S0, Alpha, Mu )
    figTi = go.Figure()
    figTi.add_trace(go.Scatter(x=t, y=ti_overtime, name='CR(t)', mode = "lines", line = dict(color = "blue", shape = "spline", smoothing = 1))) 
    figTi.update_layout(autosize=False, width= 900, height=500, title='Transmission Rate ({})')
    
    ti0 = Ti_0(x2, v1, v2, S0,  Eta, Delta )
    print('Ti_0 = ' + "{:.2E}".format(ti0))
    print('R_0 = ' + "{:.2f}".format(R_0(ti0,v1, v2, S0, Eta )))
    figTi.show()
    figCum.show()
    figDaily.show()
    

## Showing the interactions

In [63]:
RepFraction = wg.FloatSlider(description='f:',value=0.1,min=0.1, max=0.9,step=0.1) 
interact(plotsiru ,Country = ['Germany','Italy','South Korea','United Kingdom'], f= RepFraction)

interactive(children=(FloatSlider(value=0.1, description='f:', max=0.9, min=0.1), Output()), _dom_classes=('wi…

<function __main__.plotsiru(f)>